In [ ]:
!pip install roboflow

# Get Dataset

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="cJUunyRGd3YuAM3cnXbU")
project = rf.workspace("actions-players").project("players-dataset")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Players-Dataset-4 in yolov8:: 100%|██████████| 424/424 [00:00<00:00, 7282.31it/s]


In [3]:
import shutil

shutil.move('Players-Dataset-4/train',
            'Players-Dataset-4/Players-Dataset-4/train'
            )

shutil.move('Players-Dataset-4/test',
            'Players-Dataset-4/Players-Dataset-4/test'
            )

shutil.move('Players-Dataset-4/valid',
            'Players-Dataset-4/Players-Dataset-4/valid'
            )

'Players-Dataset-4/Players-Dataset-4/valid'

In [4]:
dataset.location

'/home/khalil/VAS/training/Players-Dataset-4'

# Training

In [5]:
#cleaning cash from gpu

import torch
torch.cuda.empty_cache()


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
dataset.location

'/home/khalil/VAS/training/Players-Dataset-4'

In [12]:
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=20 imgsz=420 batch=8 amp=False

New https://pypi.org/project/ultralytics/8.3.40 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 3721MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/khalil/VAS/training/Players-Dataset-4/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=420, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

# Evaluate

In [13]:
from ultralytics import YOLO

### on valid set

In [ ]:
# Load the trained model
model = YOLO('runs/detect/train10/weights/best.pt')

# Validate the model on the validation set
metrics = model.val(data='/home/khalil/VAS/training/Players-Dataset-4/data.yaml')
print(metrics)  

### on a test set

In [ ]:
# Validate the model on the test set 
metrics = model.val(data='/home/khalil/VAS/training/Players-Dataset-4/data.yaml',split='test')
print(metrics)  

# Inference

### Predections results

In [16]:
import cv2
from matplotlib import pyplot as plt

# Run YOLO inference
results = model('../image.png')  
print(results[0].boxes.data)


image 1/1 /home/khalil/VAS/training/../image.png: 256x448 12 players, 39.5ms
Speed: 1.4ms preprocess, 39.5ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 448)
tensor([[4.3205e+02, 5.0317e+02, 4.6643e+02, 6.2034e+02, 7.6612e-01, 0.0000e+00],
        [3.5227e+02, 5.4701e+02, 3.9729e+02, 6.8708e+02, 7.6568e-01, 0.0000e+00],
        [4.9186e+02, 4.6543e+02, 5.3523e+02, 5.5766e+02, 7.4310e-01, 0.0000e+00],
        [6.9974e+02, 4.2740e+02, 7.3794e+02, 5.5332e+02, 7.2838e-01, 0.0000e+00],
        [9.1596e+02, 4.7053e+02, 9.4940e+02, 5.9985e+02, 6.3586e-01, 0.0000e+00],
        [1.0695e+03, 4.9924e+02, 1.0970e+03, 6.2108e+02, 6.3280e-01, 0.0000e+00],
        [8.9861e+02, 4.6801e+02, 9.3175e+02, 5.9978e+02, 5.6972e-01, 0.0000e+00],
        [2.5026e+02, 6.0547e+02, 2.9023e+02, 7.3682e+02, 5.4193e-01, 0.0000e+00],
        [1.5719e+03, 4.5023e+02, 1.6173e+03, 5.8258e+02, 5.3597e-01, 0.0000e+00],
        [1.1661e+03, 6.1021e+02, 1.2247e+03, 7.8749e+02, 4.9445e-01, 0.0000e+00],
     

### boxes plot

In [19]:
import cv2
from matplotlib import pyplot as plt

# Load the image
image_path = '../2.png'
frame = cv2.imread(image_path)

# Convert BGR image to RGB for visualization
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# YOLO model inference
results = model(image_path)

# Extract the bounding boxes
boxes = results[0].boxes.data.cpu().numpy()  # Convert to numpy array for easier handling

# Iterate through the boxes and draw them
for box in boxes:
    x_min, y_min, x_max, y_max, confidence, class_id = box
    x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

    # Draw the rectangle on the image
    color = (0, 255, 0)  # Green for the bounding box
    thickness = 2
    cv2.rectangle(frame_rgb, (x_min, y_min), (x_max, y_max), color, thickness)

    # Optional: Add class ID and confidence
    label = f"Class {int(class_id)}, {confidence:.2f}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    text_thickness = 1
    cv2.putText(frame_rgb, label, (x_min, y_min - 10), font, font_scale, color, text_thickness)

# Display the image with bounding boxes using matplotlib
plt.imshow(frame_rgb)
plt.axis('off')
plt.show()

# Optionally, save the resulting image
cv2.imwrite("../output_with_boxes.png", cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))



image 1/1 /home/khalil/VAS/training/../2.png: 256x448 16 players, 15.5ms
Speed: 15.8ms preprocess, 15.5ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 448)


<Figure size 640x480 with 1 Axes>

True